In [12]:
using Plots
using Polynomials, SpecialPolynomials # これらの行がファイルの先頭にあることを確認してください

# LGビーム関連関数
function Rayleigh_Range(w0, lambda)
    return pi * w0^2 / lambda
end

function w_z(z, w0, zR)
    return w0 * sqrt(1 + (z/zR)^2)
end

"""
    Laguerre_L(p::Int, l::Int, x::Real)

SpecialPolynomials.jl を使用して、関連ラゲール多項式 L_p^(l)(x) を評価します。

# 引数
- `p::Int`: 動径指数 (多項式の次数)。ユーザーのクエリにおける P に相当。非負である必要があります。
- `l::Int`: 方位角指数 (パラメータ alpha)。ユーザーのクエリにおける l に相当。非負である必要があります。
- `x::Real`: 多項式を評価する点。

# 戻り値
- `::Real`: L_p^(l)(x) の値。

# エラー
- `ArgumentError`: p または l が負の場合。
"""
function Laguerre_L(p::Int, l::Int, x::Real)
    if p < 0 || l < 0 # 論理OR演算子 `||` は修正済みです
        throw(ArgumentError("指数 p および l は非負の整数でなければなりません。"))
    end

    # 型安定性と堅牢性のために、alpha パラメータが Float64 であることを保証
    # basis 関数は L_p^(l)(x) 多項式オブジェクトを返します
    L_poly = basis(Laguerre{Float64(l)}, p) # [1]

    # 多項式を点 x で評価
    return L_poly(x)
end


function LG_Spatial_Intensity(r, z, p, l, w0, lambda)
    zR = Rayleigh_Range(w0, lambda)
    current_wz = w_z(z, w0, zR)
    x_arg = 2 * r^2 / current_wz^2

    # r=0でl > 0の場合の特異点処理 (強度0) [2, 3]
    if r == 0 && l!= 0
        return 0.0
    end

    # LGビームの強度プロファイル式の一部
    radial_term_power = x_arg^abs(l)
    laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

    return (w0/current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
end

# 固定パラメータ
p_index_fixed = 3 # ここを変更してPの値を設定してください (例: 0, 1, 2, 3)
lambda_laser = 1.064e-6 # レーザーの波長 (m), 例として1064 nm
w0_beam = 1.0e-6 # ビームウェスト w0 (m), 例として1マイクロメートル
z_position = 0.0 # 焦点面 (z=0) での強度分布を描画

# プロット範囲の定義 (マイクロメートル単位)
x_min, x_max = -5e-6, 5e-6
y_min, y_max = -5e-6, 5e-6
num_points = 200 # 各軸のグリッド点数

# グリッドの作成
x_vals = range(x_min, x_max, length=num_points)
y_vals = range(y_min, y_max, length=num_points)

# 各l値のプロットを格納する配列を初期化
plots_array = []　# ここを空の配列として初期化しました！

# 描画したいLの値を設定 (例: 0, 1, 2, 3)
l_val = 0 # ここを変更してLの値を設定してください

# 強度データの計算
intensity_data = zeros(num_points, num_points)
for i in 1:num_points
    for j in 1:num_points
        x = x_vals[i]
        y = y_vals[j]
        r = sqrt(x^2 + y^2)
        intensity_data[i, j] = LG_Spatial_Intensity(r, z_position, p_index_fixed, l_val, w0_beam, lambda_laser)
    end
end

# 強度データを正規化してプロットの視覚化を改善
# 各プロット内で最大値で正規化することで、コントラストを最適化します。
normalized_intensity = intensity_data./ maximum(intensity_data)

# ヒートマップのプロットを作成し、配列に追加
p = heatmap(x_vals.* 1e6, y_vals.* 1e6, normalized_intensity',
            aspect_ratio=:equal,
            xlabel="X (µm)",
            ylabel="Y (µm)",
            title="LG($(p_index_fixed))($(l_val)) Beam Intensity (z=0 µm)",
            colorbar_title="Normalized Intensity",
            c=:viridis, # カラーマップ
            size=(400, 400)) # 個々のプロットサイズを調整
push!(plots_array, p)


# 全てのプロットを並べて表示
# layout=(1, length(plots_array)) で、プロットを1行に並べます。
plot(plots_array..., layout=(1, length(plots_array)), size=(400, 400), plot_title="LG Beam Intensity Profile (p=$(p_index_fixed), l=$(l_val))")

# プロットをファイルに保存 (オプション)
savefig("LG_beam_ppp$(p_index_fixed)_l$(l_val).png")

"C:\\Users\\mashu\\LG_beam_ppp3_l0.png"